In [1]:
import datetime
import h5py
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import tables
from tqdm import tqdm
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
all_data= pd.read_csv(r"C:\Users\ag-bahl\Desktop\\all_dataframes_stimulus_09_11.csv")
print('done all data')

done all data


C:\Users\ag-bahl\AppData\Local\Temp\ipykernel_6624\2769563268.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data= pd.read_csv(r"C:\Users\ag-bahl\Desktop\\all_dataframes_stimulus_09_11.csv")


In [3]:
all_data

,setup_index,arena_index,experiment_repeat,fish_ID,fish_name,fish_genotype,fish_age,stimulus_name,trial,start_time,...,y_position_change,distance_change,estimated_orientation_change,average_speed,interbout_interval,same_direction_as_previous_bout,start_time_absolute,end_time_absolute,start_accumulated_orientation_windowed_variance,end_accumulated_orientation_windowed_variance
0,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,3,32.899434,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-09 17:23:58.383394560+01:00,2022-11-09 17:23:58.450060544+01:00,NaN,NaN
1,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,3,34.877196,...,-0.022065,0.031111,58.890305,0.215385,1.911096,0.0,2022-11-09 17:24:00.361156352+01:00,2022-11-09 17:24:00.505599488+01:00,NaN,NaN
2,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,4,19.086126,...,0.002245,0.036542,70.626675,0.205551,44.066309,1.0,2022-11-09 17:24:44.571908864+01:00,2022-11-09 17:24:44.749685248+01:00,NaN,NaN
3,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,4,20.141673,...,-0.000932,0.007229,6.385437,0.162663,0.877771,1.0,2022-11-09 17:24:45.627456+01:00,2022-11-09 17:24:45.671900160+01:00,NaN,NaN
4,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,4,20.908333,...,0.000406,0.006540,5.680229,0.147150,0.722216,1.0,2022-11-09 17:24:46.394116352+01:00,2022-11-09 17:24:46.438560512+01:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474919,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,15.778921,...,0.000500,0.004179,0.300847,0.075214,0.433330,1.0,2022-12-06 18:37:47.147084032+01:00,2022-12-06 18:37:47.202639104+01:00,2.499966,1.164014
474920,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,15.923364,...,0.000073,0.007753,-0.624565,0.063431,0.088888,0.0,2022-12-06 18:37:47.291527168+01:00,2022-12-06 18:37:47.413748480+01:00,2.155910,1.396932
474921,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,17.212243,...,0.000142,0.010726,0.704548,0.080444,1.166657,0.0,2022-12-06 18:37:48.580405760+01:00,2022-12-06 18:37:48.713737984+01:00,2.555698,1.155737
474922,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,17.945570,...,0.000078,0.005362,-0.477687,0.096525,0.599995,0.0,2022-12-06 18:37:49.313733120+01:00,2022-12-06 18:37:49.369288192+01:00,2.203824,1.392054


In [4]:
# reorder the data according to first fish_name, then trial.
all_data.sort_values(by=['fish_name', 'trial'], inplace=True, ascending=True)
all_data

,setup_index,arena_index,experiment_repeat,fish_ID,fish_name,fish_genotype,fish_age,stimulus_name,trial,start_time,...,y_position_change,distance_change,estimated_orientation_change,average_speed,interbout_interval,same_direction_as_previous_bout,start_time_absolute,end_time_absolute,start_accumulated_orientation_windowed_variance,end_accumulated_orientation_windowed_variance
0,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,3,32.899434,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-09 17:23:58.383394560+01:00,2022-11-09 17:23:58.450060544+01:00,NaN,NaN
1,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,3,34.877196,...,-0.022065,0.031111,58.890305,0.215385,1.911096,0.0,2022-11-09 17:24:00.361156352+01:00,2022-11-09 17:24:00.505599488+01:00,NaN,NaN
2,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,4,19.086126,...,0.002245,0.036542,70.626675,0.205551,44.066309,1.0,2022-11-09 17:24:44.571908864+01:00,2022-11-09 17:24:44.749685248+01:00,NaN,NaN
3,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,4,20.141673,...,-0.000932,0.007229,6.385437,0.162663,0.877771,1.0,2022-11-09 17:24:45.627456+01:00,2022-11-09 17:24:45.671900160+01:00,NaN,NaN
4,0,0,0,16,2022-11-09_17-20-25_setup0_arena0,WT,5 dpf,60_plaid_0,4,20.908333,...,0.000406,0.006540,5.680229,0.147150,0.722216,1.0,2022-11-09 17:24:46.394116352+01:00,2022-11-09 17:24:46.438560512+01:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474919,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,15.778921,...,0.000500,0.004179,0.300847,0.075214,0.433330,1.0,2022-12-06 18:37:47.147084032+01:00,2022-12-06 18:37:47.202639104+01:00,2.499966,1.164014
474920,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,15.923364,...,0.000073,0.007753,-0.624565,0.063431,0.088888,0.0,2022-12-06 18:37:47.291527168+01:00,2022-12-06 18:37:47.413748480+01:00,2.155910,1.396932
474921,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,17.212243,...,0.000142,0.010726,0.704548,0.080444,1.166657,0.0,2022-12-06 18:37:48.580405760+01:00,2022-12-06 18:37:48.713737984+01:00,2.555698,1.155737
474922,1,6,0,152,2022-12-06_17-56-31_setup1_arena6,WT,5 dpf,60_plaid_0,41,17.945570,...,0.000078,0.005362,-0.477687,0.096525,0.599995,0.0,2022-12-06 18:37:49.313733120+01:00,2022-12-06 18:37:49.369288192+01:00,2.203824,1.392054


In [5]:
# split the data into optimum gray and stimulus.

In [6]:
# then get a dataframe with streak lengths, for:
# 1. All data
# 2. Optimum gray only
# 3. Plaids, or whatever stimulus.


In [7]:
# then get the values, from Ashrit's functions as you got berfore, mean and sem through a trial

In [8]:
# Then we get similar to Ashrit's function, every other variable that I consider important

In [9]:
# Then histograms, loads hahaha - the turning angle distributions for each configuration

In [10]:
# then measure the streak length, and first a histogram, then the CDF and PDF whatever.
# and then for each combination of stimuli, you add this after you get 8 direction data...

In [11]:
all_data.columns

Index(['setup_index', 'arena_index', 'experiment_repeat', 'fish_ID',
       'fish_name', 'fish_genotype', 'fish_age', 'stimulus_name', 'trial',
       'start_time', 'start_contour_area', 'start_x_position',
       'start_y_position', 'start_accumulated_path',
       'start_accumulated_orientation', 'end_time', 'end_contour_area',
       'end_x_position', 'end_y_position', 'end_accumulated_path',
       'end_accumulated_orientation', 'duration', 'x_position_change',
       'y_position_change', 'distance_change', 'estimated_orientation_change',
       'average_speed', 'interbout_interval',
       'same_direction_as_previous_bout', 'start_time_absolute',
       'end_time_absolute', 'start_accumulated_orientation_windowed_variance',
       'end_accumulated_orientation_windowed_variance'],
      dtype='object')

In [ ]:
def plot_pref(bin_df, pref='percentage_left', time_marker=None):
    num_fish = len(bin_df.index.unique('fish_ID'))
    t_stamp = bin_df.index.unique('binned_time')
    bin_df = bin_df.reset_index()

    mean_df = bin_df.groupby(['stimulus_name', 'binned_time']).mean()
    sem_df = bin_df.groupby(['stimulus_name', 'binned_time']).std() / (num_fish) ** 0.5

    legend = []
    for stim in mean_df.index.unique('stimulus_name'):
        legend.append(stim)

        plot_mean_df = mean_df.xs(stim, level='stimulus_name')
        plot_sem_df = sem_df.xs(stim, level='stimulus_name')

        if pref == 'preference_index':
            plt.plot(plot_mean_df.preference_index, marker='o', linewidth=1, markersize=2)
            plt.fill_between(t_stamp, plot_mean_df.preference_index + plot_sem_df.preference_index,
                             plot_mean_df.preference_index - plot_sem_df.preference_index,
                             alpha=0.25, label='_no_legend_')
        elif pref == 'percentage_left':
            plt.plot(plot_mean_df.percentage_left, marker='o', linewidth=1, markersize=2)
            plt.fill_between(t_stamp, plot_mean_df.percentage_left + plot_sem_df.percentage_left,
                             plot_mean_df.percentage_left - plot_sem_df.percentage_left,
                             alpha=0.25, label='_nolegend_')
            plt.axhline(y=50, color='grey', linestyle='--', alpha=0.2, label='_nolegend_')
            plt.ylim(0, 100)
            plt.ylabel('Percentage left')
            plt.xlabel('Time (s)')
    if time_marker != None:
        for t in time_marker:
            plt.axvline(x=t, color='grey', linestyle='--', alpha=0.4, label='_nolegend_')

    plt.legend(legend)
    plt.show()

    return plt

In [ ]:
def plot_orientation(label_df, bins=50):
    c = 0.7
    for stim in label_df.index.unique('stimulus_name'):
        temp_df = label_df.xs(stim, level='stimulus_name')

        plt.hist(temp_df.orientation_change, bins=bins, range=[-180, 180],
                 edgecolor='black', alpha=c - 0.1, )

    plt.show()

    return plt


plot_orientation(label_df, bins=50)